### Part 1  Leaves Recognition using CNN
    We do the machine learning work use CNN method. the trainset We use is a csv file. And there are 192 parameters in the csv file for each leaves. and we will train it with several methods and compare the respectively accuracy with each other and find the best model.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,Convolution2D
from keras.optimizers import SGD
from keras.utils import np_utils

C:\Users\lanwe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('100 leaves plant species/train0.csv')
test = pd.read_csv('100 leaves plant species/test01.csv')

Encode train and test in order

In [3]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, classes


In [4]:
train, labels, test, classes = encode(train, test)

Use standerdscaler to remove the mean and normalize the variance.
Split train data into train and validation

In [5]:
# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]
    

nb_features = 64 # number of features per features type (shape, texture, margin)   
nb_class = len(classes)

Reshape data

In [6]:
# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

### Train model

Single convolutional layer

    Activation method: relu;
    Train method: softmax;
    Loss: categorical_crossentropty;
    Opimizer: sgd;

In [7]:
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(64, 3), filters=512, kernel_size=1)`
  after removing the cwd from sys.path.
C:\Users\lanwe\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1116 samples, validate on 124 samples
Epoch 1/15
1116/1116 [==============================] - 54s 48ms/step - loss: 4.4955 - acc: 0.0797 - val_loss: 4.3030 - val_acc: 0.2903
Epoch 2/15
1116/1116 [==============================] - 52s 47ms/step - loss: 4.1240 - acc: 0.2841 - val_loss: 3.8020 - val_acc: 0.3629
Epoch 3/15
1116/1116 [==============================] - 53s 47ms/step - loss: 3.4783 - acc: 0.4579 - val_loss: 2.9814 - val_acc: 0.6210
Epoch 4/15
1116/1116 [==============================] - 52s 47ms/step - loss: 2.4692 - acc: 0.6819 - val_loss: 1.8897 - val_acc: 0.7742
Epoch 5/15
1116/1116 [==============================] - 52s 47ms/step - loss: 1.4162 - acc: 0.8253 - val_loss: 1.1243 - val_acc: 0.8629
Epoch 6/15
1116/1116 [==============================] - 52s 47ms/step - loss: 0.7641 - acc: 0.9167 - val_loss: 0.7441 - val_acc: 0.9032
Epoch 7/15
1116/1116 [==============================] - 54s 48ms/step - loss: 0.4750 - acc: 0.9462 - val_loss: 0.5711 - val_acc: 0.8952
E

### Change train method to sigmoid
    Therefore, it is especially used for models where we have to predict the probability as an output.Since probability of anything exists only between the range of 0 and 1, sigmoid is the right choice.   

In [7]:
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('sigmoid'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(64, 3), filters=512, kernel_size=1)`
  after removing the cwd from sys.path.
C:\Users\lanwe\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1116 samples, validate on 124 samples
Epoch 1/15
1116/1116 [==============================] - 54s 49ms/step - loss: 4.5821 - acc: 0.0323 - val_loss: 4.5418 - val_acc: 0.1855
Epoch 2/15
1116/1116 [==============================] - 52s 47ms/step - loss: 4.5172 - acc: 0.1577 - val_loss: 4.4757 - val_acc: 0.3387
Epoch 3/15
1116/1116 [==============================] - 52s 47ms/step - loss: 4.4473 - acc: 0.3065 - val_loss: 4.4015 - val_acc: 0.4274
Epoch 4/15
1116/1116 [==============================] - 53s 47ms/step - loss: 4.3665 - acc: 0.4274 - val_loss: 4.3148 - val_acc: 0.5161
Epoch 5/15
1116/1116 [==============================] - 53s 47ms/step - loss: 4.2686 - acc: 0.5197 - val_loss: 4.2056 - val_acc: 0.5726
Epoch 6/15
1116/1116 [==============================] - 52s 47ms/step - loss: 4.1297 - acc: 0.5923 - val_loss: 4.0350 - val_acc: 0.6210
Epoch 7/15
1116/1116 [==============================] - 52s 47ms/step - loss: 3.8626 - acc: 0.6631 - val_loss: 3.6246 - val_acc: 0.6855
E

### Loss method changed to binary_crossentropy
    but the loss method is not suitable for our dataset, because the scale and the shape of our dataset
    the accuracy computed with the Keras method "evaluate" is just plain wrong when using binary_crossentropy with more than 2 labels

In [7]:
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=512, kernel_size=1, input_shape=(64, 3))`
  after removing the cwd from sys.path.
C:\Users\lanwe\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1116 samples, validate on 124 samples
Epoch 1/15
1116/1116 [==============================] - 64s 57ms/step - loss: 0.0560 - acc: 0.9900 - val_loss: 0.0560 - val_acc: 0.9900
Epoch 2/15
1116/1116 [==============================] - 71s 63ms/step - loss: 0.0560 - acc: 0.9900 - val_loss: 0.0560 - val_acc: 0.9900
Epoch 3/15
1116/1116 [==============================] - 70s 63ms/step - loss: 0.0560 - acc: 0.9900 - val_loss: 0.0560 - val_acc: 0.9900
Epoch 4/15
1116/1116 [==============================] - 71s 63ms/step - loss: 0.0559 - acc: 0.9900 - val_loss: 0.0560 - val_acc: 0.9900
Epoch 5/15
1116/1116 [==============================] - 72s 64ms/step - loss: 0.0559 - acc: 0.9900 - val_loss: 0.0559 - val_acc: 0.9900
Epoch 6/15
1116/1116 [==============================] - 74s 67ms/step - loss: 0.0559 - acc: 0.9900 - val_loss: 0.0559 - val_acc: 0.9900
Epoch 7/15
1116/1116 [==============================] - 71s 64ms/step - loss: 0.0559 - acc: 0.9900 - val_loss: 0.0559 - val_acc: 0.9900
E

Reduce epoch from 15 to 8, see  if we can get same accuracy, and the answer is no.

In [7]:
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

nb_epoch = 8
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(64, 3), filters=512, kernel_size=1)`
  after removing the cwd from sys.path.
C:\Users\lanwe\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1116 samples, validate on 124 samples
Epoch 1/8
1116/1116 [==============================] - 56s 50ms/step - loss: 4.5169 - acc: 0.0789 - val_loss: 4.3393 - val_acc: 0.3306
Epoch 2/8
1116/1116 [==============================] - 54s 49ms/step - loss: 4.1789 - acc: 0.2912 - val_loss: 3.8778 - val_acc: 0.4355
Epoch 3/8
1116/1116 [==============================] - 54s 49ms/step - loss: 3.5785 - acc: 0.4731 - val_loss: 3.1012 - val_acc: 0.5968
Epoch 4/8
1116/1116 [==============================] - 54s 49ms/step - loss: 2.6146 - acc: 0.6703 - val_loss: 2.0194 - val_acc: 0.7581
Epoch 5/8
1116/1116 [==============================] - 55s 49ms/step - loss: 1.5097 - acc: 0.8423 - val_loss: 1.1759 - val_acc: 0.8790
Epoch 6/8
1116/1116 [==============================] - 54s 49ms/step - loss: 0.8113 - acc: 0.9149 - val_loss: 0.7711 - val_acc: 0.8871
Epoch 7/8
1116/1116 [==============================] - 54s 49ms/step - loss: 0.4874 - acc: 0.9409 - val_loss: 0.5695 - val_acc: 0.8790
Epoch 8/

### Change optimizer from sgd to adam
The Adaptive Moment Estimation is essentially RMSprop with momentum term, which USES the first Moment Estimation of the gradient and the second-order Moment Estimation to dynamically adjust the learning rate of each parameter.The advantage of Adam is that after the offset correction, each iteration learning rate has a certain range, which makes the parameter relatively stable.

In [7]:
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))


y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=512, kernel_size=1, input_shape=(64, 3))`
  after removing the cwd from sys.path.
C:\Users\lanwe\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1116 samples, validate on 124 samples
Epoch 1/15
1116/1116 [==============================] - 110s 98ms/step - loss: 2.4553 - acc: 0.4749 - val_loss: 0.9471 - val_acc: 0.7419
Epoch 2/15
1116/1116 [==============================] - 108s 96ms/step - loss: 0.3486 - acc: 0.9023 - val_loss: 0.5368 - val_acc: 0.8306
Epoch 3/15
1116/1116 [==============================] - 108s 97ms/step - loss: 0.1140 - acc: 0.9713 - val_loss: 0.5350 - val_acc: 0.8306
Epoch 4/15
1116/1116 [==============================] - 108s 97ms/step - loss: 0.2570 - acc: 0.9471 - val_loss: 0.6428 - val_acc: 0.8629
Epoch 5/15
1116/1116 [==============================] - 108s 97ms/step - loss: 0.1240 - acc: 0.9686 - val_loss: 0.5016 - val_acc: 0.8629
Epoch 6/15
1116/1116 [==============================] - 108s 97ms/step - loss: 0.0777 - acc: 0.9776 - val_loss: 0.6778 - val_acc: 0.8548
Epoch 7/15
1116/1116 [==============================] - 108s 97ms/step - loss: 0.1558 - acc: 0.9686 - val_loss: 0.5772 - val_acc: 0

when we use the epoch 15, softmax , sgd categorical_crossentropy we can get the best accuracy about 95 percent. Although when the loss moehod is Binary_crossentropy,we can get 99 percent accuracy,but it may not suitable for our dataset because we cannot make data binary.

    reference: CNN example:https://www.kaggle.com/tobikaggle/nn-through-keras-copied-mod-shuffle

    CNN: https://github.com/keras-team/keras/blob/master/examples/imdb_cnn.py

    CNN: https://www.kaggle.com/tonypoe/keras-cnn-example?scriptVersionId=589403

 Copyright <2018> <Weiyi Lan YiQun Xu Yang Zong>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.